In [1]:
import torch

path_checkpoints = 'Evaluation_from_checkpoints/checkpoints/model_checkpoints/1_deeplabv3_resnet101_adagrad.pth'

try:
    checkpoint = torch.load(path_checkpoints, map_location=torch.device('cuda:0'))
    print("Checkpoint loaded successfully.")
    print(checkpoint.keys())
except RuntimeError as e:
    print("Failed to load checkpoint:", e)

Checkpoint loaded successfully.
dict_keys(['epoch', 'model_state_dict'])


In [2]:
import src.architectures as architectures
from torch.utils.data import DataLoader


/home/corbusier/miniconda3/envs/seg_comp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pasta com os checkpoints (*.pth):  /home/corbusier/development/compara_segmentadores_torch_/checkpoints


In [3]:
model = architectures.segmentation.deeplabv3_resnet101(pretrained=False)

/home/corbusier/miniconda3/envs/seg_comp/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/corbusier/miniconda3/envs/seg_comp/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from pycocotools.coco import COCO

class COCOMultiLabelSegmentationDataset(Dataset):
    def __init__(self, root_dir, annotation_file, include_patterns=None, transform=None):
        self.root_dir = root_dir
        self.coco = COCO(annotation_file)
        
        # Filter image IDs based on the include_patterns
        self.image_ids = []
        for image_id in self.coco.imgs.keys():
            img_info = self.coco.loadImgs(image_id)[0]
            img_path = os.path.join(self.root_dir, img_info['file_name'])
            if os.path.exists(img_path):
                if include_patterns:
                    if any(pattern in img_info['file_name'] for pattern in include_patterns):
                        self.image_ids.append(image_id)
                else:
                    self.image_ids.append(image_id)
        
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        ann_ids = self.coco.getAnnIds(imgIds=image_id)
        anns = self.coco.loadAnns(ann_ids)
        
        img_info = self.coco.loadImgs(image_id)[0]
        img_path = os.path.join(self.root_dir, img_info['file_name'])
        image = Image.open(img_path).convert('RGB')

        # Create an empty mask
        mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

        for ann in anns:
            mask = np.maximum(mask, self.coco.annToMask(ann) * (ann['category_id'] + 1))

        mask = Image.fromarray(mask)
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return image, mask

# Define any transformations (e.g., resizing, normalization) to apply to the images and masks
image_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

mask_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

# Create an instance of the dataset
root_dir = '/home/corbusier/development/compara_segmentadores_torch_/data_512_UCDB_UPS/all/imagens'
annotation_file = '/home/corbusier/development/compara_segmentadores_torch_/data_512_UCDB_UPS/annotations_coco_json/_annotations.coco.json'
# include_patterns = ['UPS', 'UCDB']
include_patterns = ['UPS']

dataset = COCOMultiLabelSegmentationDataset(root_dir=root_dir, annotation_file=annotation_file, include_patterns=include_patterns, transform=image_transform)

# Create a DataLoader for your dataset
val_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# Example usage: iterate over the DataLoader
for images, masks in val_loader:
    # Perform your evaluation here
    pass


loading annotations into memory...
Done (t=0.55s)
creating index...
index created!


In [12]:
def evaluate(model, dataloader, device, num_classes):
    all_preds = []
    all_labels = []

    model.to(device)
    model.eval()
    
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)
            labels = labels.squeeze(1)  # Remove the extra dimension in labels
            outputs = model(images)['out']
            preds = torch.argmax(outputs, dim=1)  # Convert to discrete class labels
            
            # Print shapes for debugging
            print(f'Batch {i+1}: images shape: {images.shape}, labels shape: {labels.shape}, preds shape: {preds.shape}')
            
            # Move to CPU and convert to numpy arrays
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # Convert to numpy arrays
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    
    # Print shapes for debugging
    print(f'All preds shape: {all_preds.shape}, all labels shape: {all_labels.shape}')
    print(f'Unique labels: {np.unique(all_labels)}, unique preds: {np.unique(all_preds)}')
    
    return all_preds, all_labels


In [13]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Number of classes (background, vegetation, grass)
num_classes = 3

# Get predictions and true labels
preds, labels = evaluate(model, val_loader, device, num_classes)

Batch 1: images shape: torch.Size([16, 3, 512, 512]), labels shape: torch.Size([16, 512, 512]), preds shape: torch.Size([16, 512, 512])
Batch 2: images shape: torch.Size([16, 3, 512, 512]), labels shape: torch.Size([16, 512, 512]), preds shape: torch.Size([16, 512, 512])
Batch 3: images shape: torch.Size([16, 3, 512, 512]), labels shape: torch.Size([16, 512, 512]), preds shape: torch.Size([16, 512, 512])
Batch 4: images shape: torch.Size([16, 3, 512, 512]), labels shape: torch.Size([16, 512, 512]), preds shape: torch.Size([16, 512, 512])
Batch 5: images shape: torch.Size([16, 3, 512, 512]), labels shape: torch.Size([16, 512, 512]), preds shape: torch.Size([16, 512, 512])
Batch 6: images shape: torch.Size([16, 3, 512, 512]), labels shape: torch.Size([16, 512, 512]), preds shape: torch.Size([16, 512, 512])
Batch 7: images shape: torch.Size([16, 3, 512, 512]), labels shape: torch.Size([16, 512, 512]), preds shape: torch.Size([16, 512, 512])
Batch 8: images shape: torch.Size([16, 3, 512, 5

In [14]:
# Flatten arrays
preds_flat = preds.flatten()
labels_flat = labels.flatten()

# Print flattened shapes for debugging
print(f'Flattened preds shape: {preds_flat.shape}, flattened labels shape: {labels_flat.shape}')

# Filter out invalid labels (e.g., labels not in [0, num_classes-1])
valid_mask = (labels_flat >= 0) & (labels_flat < num_classes)
preds_flat = preds_flat[valid_mask]
labels_flat = labels_flat[valid_mask]

# Ensure labels are integers
preds_flat = preds_flat.astype(int)
labels_flat = labels_flat.astype(int)

# Calculate precision, recall, and F1-score
precision = precision_score(labels_flat, preds_flat, average='macro', labels=np.arange(num_classes))
recall = recall_score(labels_flat, preds_flat, average='macro', labels=np.arange(num_classes))
f1 = f1_score(labels_flat, preds_flat, average='macro', labels=np.arange(num_classes))

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

# Calculate class-specific metrics
class_precision = precision_score(labels_flat, preds_flat, average=None, labels=np.arange(num_classes))
class_recall = recall_score(labels_flat, preds_flat, average=None, labels=np.arange(num_classes))
class_f1 = f1_score(labels_flat, preds_flat, average=None, labels=np.arange(num_classes))

print(f'Class Precision: {class_precision}')
print(f'Class Recall: {class_recall}')
print(f'Class F1-score: {class_f1}')

# Calculate mIoU
def mean_iou(pred, true, num_classes):
    ious = []
    for cls in range(num_classes):
        pred_cls = pred == cls
        true_cls = true == cls
        intersection = np.logical_and(pred_cls, true_cls).sum()
        union = np.logical_or(pred_cls, true_cls).sum()
        if union == 0:
            ious.append(float('nan'))  # If there is no ground truth, do not include in mean IoU
        else:
            iou = intersection / union
            ious.append(iou)
    return np.nanmean(ious)

miou = mean_iou(preds, labels, num_classes)
print(f'mIoU: {miou}')

Flattened preds shape: (197656576,), flattened labels shape: (197656576,)


/home/corbusier/miniconda3/envs/seg_comp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/corbusier/miniconda3/envs/seg_comp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/corbusier/miniconda3/envs/seg_comp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"

Precision: 0.3333333333333333
Recall: 9.520553467444463e-05
F1-score: 0.0001903567003145923


/home/corbusier/miniconda3/envs/seg_comp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/corbusier/miniconda3/envs/seg_comp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/corbusier/miniconda3/envs/seg_comp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"

Class Precision: [1. 0. 0.]
Class Recall: [0.00028562 0.         0.        ]
Class F1-score: [0.00057107 0.         0.        ]
mIoU: 6.994274232775805e-05


: 